#  A Simple CGE Model with two sectors and no intermediate input

In [1]:
using JuMP, Complementarity, DataFrames, PATHSolver
PATHSolver.c_api_License_SetString("2830898829&Courtesy&&&USR&45321&5_1_2021&1000&PATH&GEN&31_12_2025&0_0_0&6000&0_0")

1

In [2]:
sec = ["sec1", "sec2"]
sc = [1, 2]
sam = [
    missing missing missing missing 12      12
    missing missing missing missing 21      21
    9       7       missing missing missing 16
    3       14      missing missing missing 17
    missing missing 16      17      missing 33
    12      21      16      17      33      missing
]
samList = ["sec1", "sec2", "lab", "cap", "hh", "total"]

6-element Vector{String}:
 "sec1"
 "sec2"
 "lab"
 "cap"
 "hh"
 "total"

In [3]:
Q0 = sam[6, 1:2]
P0 = [1, 1]
LD0 = sam[3, 1:2]
KD0 = sam[4, 1:2]
LS = sum(LD0)[1]
KS = sum(KD0)[1]
WL0 = 1
WK0 = 1
QH0 = sam[1:2, 5]
Y0 = WL0 * LS + WK0 * KS
rho = 0.75

0.75

In [4]:
# 校准生产函数参数
al = LD0 ./ (LD0 + KD0)
ak = KD0 ./ (LD0 + KD0)
alphah = P0 .* QH0 / Y0

2-element Vector{Float64}:
 0.36363636363636365
 0.6363636363636364

# 3. Generate CGE Model

In [5]:
function solve_cge()
    m = MCPModel()

    # set_optimizer_attribute(m, GAMS.ModelType(), "MCP")

    @variables m begin
        P[i = sc], (start = P0[i])
        WK, (start = WK0) 
        WL, (start = WL0)
        Q[i = sc], (start = Q0[i])
        LD[i = sc], (start = LD0[i])
        KD[i = sc], (start = KD0[i])
        Y, (start = Y0)
        QH[i = sc], (start = QH0[i])
    end

    fix(WK, 1.0)
    # production function - CES
    @mapping(m, eqL[i in sc], LD[i] * WL ^ rho - al[i] * Q[i] * P[i] ^ rho)
    @complementarity(m, eqL, LD)

    @mapping(m, eqK[i in sc], KD[i] * WK ^ rho - ak[i] * Q[i] * P[i] ^ rho)
    @complementarity(m, eqK, KD)

    @mapping(m, eqP[i in sc], al[i] * WL ^ (1 - rho) + ak[i] * WK ^ (1 - rho) - P[i] ^ (1 - rho))
    @complementarity(m, eqP, P)

    # income distribution
    @mapping(m, eqY, Y - (WL * LS + WK * KS))
    @complementarity(m, eqY, Y)

    # consumption structure
    @mapping(m, eqQH[i in sc], alphah[i] * Y - P[i] * QH[i])
    @complementarity(m, eqQH, QH)

    # market clearance: goods, production factors
    @mapping(m, eqQ[i in sc], Q[i] - QH[i])
    @complementarity(m, eqQ, Q)

    @mapping(m, eqWK, KS - sum(KD[i] for i in sc))
    @complementarity(m, eqWK, WK)

    @mapping(m, eqWL, LS - sum(LD[i] for i in sc))
    @complementarity(m, eqWL, WL)

    # Model Solver
    status = solveMCP(m; convergence_tolerance=1e-8, output="yes", time_limit=3000)
    @show result_value.(WL)
end

solve_cge (generic function with 1 method)

In [6]:
solve_cge()

Reading options file /var/folders/0g/kx5tp84d27bbb7cl7_cx9_hm0000gn/T/jl_hiB7fU
 > convergence_tolerance 1.0e-8
 > output yes
 > time_limit 3000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 09:58:07 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris
Preprocessed size   : 12

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             0.0000e+00             0.0e+00 (eqP[1])

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     1     1 0.0000e+00           I 0.0e+00 0.0e+00 (eqP[1])

Major Iterations. . . . 0
Minor Iterations. . . . 0
Restarts. . . . . . . . 0
Crash Iterations. . . . 0
Gradient Steps. . . . . 0
Function Evaluations. . 1
Gradient Evaluations. . 1
Basis Time. . . . . . . 0.000003
Total Time. . . . . . . 0.048750
Residual. . . . . . . . 0.000000e+00
Postsolved residual: 0.0000e+00
result_value.(WL) = 1.0


1.0

# second experiment by adjusting the supply of labor and capital

In [7]:
LS = sum(LD0) * 2
solve_cge()

Reading options file /var/folders/0g/kx5tp84d27bbb7cl7_cx9_hm0000gn/T/jl_CPLTbJ
 > convergence_tolerance 1.0e-8
 > output yes
 > time_limit 3000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 09:58:07 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris
Preprocessed size   : 12

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             2.2627e+01             0.0e+00 (eqWL)
    1     6     0    12 2.0316e+01  1.7e-01    0.0e+00 (eqWL)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     7     2 2.0316e+01           I 0.0e+00 1.3e+01 (eqWL)
    1     1    16     3 2.2591e+01  2.3e-02 SB 0.0e+00 1.3e+01 (eqWL)
    2     1    17     4 5.5091e+00  1.0e+00 SO 0.0e+00 3.7e+00 (eqQH[1])
    3     1    18     5 5.4552e+00  1.0e+00 SO 0.0e+00 3.1e+00 (eqL[1])
    4     1    19     6 1.9489e+00  1.0e+00 SO 0.0e+00 1.1e+00 (eqQH[2])


0.41628360784400903